# Reference :
- https://learn.deeplearning.ai/courses/multi-ai-agent-systems-with-crewai

# Overview - Financial Analysis System

- This notebook shows a multi-agent Financial Analysis system. It has the following agents:
    1. Data analyst
    2. Trading strategist
    3. Trading advisor / execution
    4. Risk advisor
- It also shows how to make agents collaborate with each other using Hierarchical design pattern (i.e. Manager delegating work) in CrewAI using Process class.

# Setup
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo' # 'gpt-4-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to ensure that the market data for AAPL is continuously monitored and analyzed in order to identify trends and predict market movements accurately.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL", "context": "The task involves using statistical modeling and machine learning to identify trends and predict market movements for AAPL"}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict market movements accurately.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, New

Final Answer: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance Oops, something went wrong Skip to navigation Skip to main content Skip to right column News Today's news US Politics World Weather Climate change Health Wellness Mental health Sexual health Dermatology Oral health Hair loss Foot health Nutrition Healthy eating Meal delivery Weight loss Vitamins and supplements Fitness Equipment Exercise Women’s health Sleep Healthy aging Hearing Mobility Science Originals The 360 Newsletters Games Life Health Wellness Nutrition Fitness Healthy aging Mental health Sleep Your body Dermatology Children's health Foot health Hair loss Hearing aids Mental health Mobility Nutrition Personal care Sleep Women's health Home and garden Bedding Cleaning Gardening Kitchen Outdoor Pets Tech Accessories Audio Auto Computers Phones Smart home TVs Gift ideas Stores Amazon Best Buy Home Depot Macy's Nordstrom Target Walmart Wayfair Shopping Guides Best non-toxic cutting boards Best heate

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
AAPL Apple Inc. 245.40 +0.05%
Summary News Research Chart Community Statistics Historical Data Profile Financials Analysis Options Holders Sustainability
NasdaqGS - Nasdaq Real Time Price • USD
Apple Inc. (AAPL)
Follow View Analysis
Time to act on AAPL?
245.27 -8.77 (-3.45%)
At close: October 10 at 4:00:02 PM EDT
245.40 +0.13 +(0.05%)
After hours: October 10 at 7:59:58 PM EDT
Time to act on AAPL? Chart Range Bar 1D 5D -4.94% 1M 8.15% 6M 28.80% YTD -2.06% 1Y 7.09% 5Y 109.69% All 190,997.66%
Key Events
Mountain
Advanced Chart
Loading chart for AAPL
Previous Close 254.04
Open 255.04
Bid 232.30 x 100
Ask 257.91 x 100
Day's Range 244.57 - 256.38
52 Week Range 169.21 - 260.10
Volume 61,156,139
Avg. Volume 54,836,700
Market Cap (intraday) 3.64T
Beta (5Y Monthly) 1.09
PE Ratio (TTM) 37.16
EPS (TTM) 6.60
Earnings Date Oct 30, 2025
Forward Dividend & Yield 1.04 (0.42%)
Ex-Dividend Date Aug 11, 2025
1y Target Est 

I need to develop and refine trading strategies for AAPL based on the insights from the Data Analyst and user-defined risk tolerance, considering trading preferences like Day Trading.

Action: Delegate work to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "task": "Develop trading strategies for AAPL based on Data Analyst insights and user-defined risk tolerance, considering Day Trading preferences", "context": "AAPL stock price, market trends, user risk tolerance, and trading preferences"}

> Entering new CrewAgentExecutor chain...
I need to gather insights from the Data Analyst Agent and then develop trading strategies for AAPL based on the information provided.

Action: Search the internet
Action Input: {"search_query": "AAPL stock price trends"} 


Search results: Title: Apple Inc. (AAPL) Stock Historical Prices & Data - Yahoo Finance
Link: https://finance.yahoo.com/quote/AAPL/history/
Snippet: Discover historical prices for AAPL stock on Yahoo Finance. View dai

I need to gather more information from the Trading Strategy Developer and the Data Analyst to ensure we have all the necessary details to determine the best execution methods for AAPL.

Action: Ask question to co-worker
Action Input: {"coworker": "Trading Strategy Developer", "question": "Can you provide detailed execution plans suggesting how and when to execute trades for AAPL based on the developed trading strategies?", "context": "I have delegated the task of developing trading strategies for AAPL to you based on insights from the Data Analyst and user-defined risk tolerance, considering Day Trading preferences."}

> Entering new CrewAgentExecutor chain...
I should start by gathering relevant information on AAPL and the current market conditions to develop effective trading strategies for day trading.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance Oops, someth

Final Answer: Based on the information gathered from the Yahoo Finance website for AAPL, I can see that Apple Inc. is currently trading at $245.40 with a slight increase of 0.05%. The stock has had a mixed performance over different time frames, with a YTD return of -2.06% and a 1-year return of 7.09%. The company's market cap is at 3.64T, showing its significant size in the market.

To develop effective trading strategies for day trading AAPL, I would recommend considering the historical performance of the stock, current market trends, and any upcoming events such as earnings reports. It is essential to set clear entry and exit points, use technical analysis indicators, and closely monitor the stock's price action throughout the trading day. Additionally, it would be beneficial to implement risk management strategies to protect against potential losses.

Overall, with a thorough analysis of AAPL's performance and market conditions, combined with a disciplined trading approach, it is p

Thought: 
I have gathered information on AAPL's current market trends, historical performance, and upcoming events. Now, I need to analyze this data to evaluate the risks associated with proposed trading strategies and execution plans.

Final Answer:
Title: Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance
Link: https://finance.yahoo.com/quote/AAPL/
Snippet: Find the latest Apple Inc. (AAPL) stock quote, history, news and other vital information to help you with your stock trading and investing.

Title: Apple (AAPL) Stock Price, Quote, News & History - Nasdaq
Link: https://www.nasdaq.com/market-activity/stocks/aapl
Snippet: Discover real-time Apple Inc. Common Stock (AAPL) stock prices, quotes, historical data, news, and Insights for informed trading and investment decisions.

Title: NASDAQ:AAPL Stock Price — TradingView - Apple Inc.
Link: https://www.tradingview.com/symbols/NASDAQ-AAPL/
Snippet: The current price of AAPL is 245.27 USD — it has decreased by −3.45% in

- Display the final result as Markdown.

In [16]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for Trading Strategies and Execution Plans for AAPL:

Introduction:
The purpose of this report is to provide a comprehensive risk analysis and mitigation strategies for the proposed trading strategies and execution plans for AAPL. The analysis is based on the context provided and aims to ensure the team's success in effectively executing trades for AAPL while maximizing profitability and managing risk.

Risk Analysis:
1. Market Risk:
- Fluctuations in the stock market may impact the execution of trading strategies.
- Mitigation Strategy: Implement risk management strategies such as stop-loss orders to protect against potential losses due to market volatility.

2. Operational Risk:
- Technical issues or errors in trade execution platforms may lead to missed opportunities.
- Mitigation Strategy: Regularly monitor the trade execution platforms and have contingency plans in place to address any operational issues promptly.

3. Financial Risk:
- Changes in interest rates or economic conditions may affect the financial performance of AAPL.
- Mitigation Strategy: Diversify the portfolio to reduce exposure to specific financial risks and stay informed about economic indicators that may impact AAPL's stock price.

4. Regulatory Risk:
- Changes in regulations or legal requirements may impact the trading environment for AAPL.
- Mitigation Strategy: Stay updated on regulatory changes and ensure compliance with all relevant laws and regulations to mitigate regulatory risks.

5. Reputational Risk:
- Negative publicity or public perception may impact the stock price of AAPL.
- Mitigation Strategy: Monitor public sentiment and have a crisis communication plan in place to address any reputational risks effectively.

Mitigation Strategies:
1. Implement a diversified portfolio to reduce exposure to specific risks.
2. Utilize risk management tools such as stop-loss orders and hedging strategies.
3. Stay informed about market trends, economic indicators, and regulatory changes.
4. Continuously evaluate and adjust trading strategies based on market conditions and performance.

Conclusion:
By following the detailed risk analysis and implementing the suggested mitigation strategies, the team can effectively execute trades for AAPL while managing risks and maximizing profitability. It is crucial to remain proactive, informed, and adaptable in the dynamic trading environment to achieve success in trading AAPL.